USing Deepseek to get the score

In [ ]:
DEEPSEEK_API_KEY = "sk-f571f15d94614c749dc2059f8f72a728"
client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

NameError: name 'OpenAI' is not defined

In [ ]:
import pandas as pd
import os
import time

# Load transaction dataset
file_path = "/content/drive/MyDrive/285project/train.csv"
df = pd.read_csv(file_path)

# Create a unique Transaction ID column
df["Transaction ID"] = df.index

# Output file path
deepseek_score_path = "/content/drive/MyDrive/285project/deepseek_scores.csv"

# Load existing scores if available
if os.path.exists(deepseek_score_path):
    processed_scores = pd.read_csv(deepseek_score_path)
    processed_ids = set(processed_scores["Transaction ID"])
else:
    processed_scores = pd.DataFrame(columns=["Transaction ID", "DeepSeek Score"])
    processed_ids = set()

# Function to get DeepSeek score for a single transaction
def get_deepseek_score(transaction):
    prompt = f"""
    Transaction Details:
    - Timestamp: {transaction['Timestamp']}
    - Sender Bank ID: {transaction['From Bank']}
    - Sender Account: {transaction['Account']}
    - Receiver Bank ID: {transaction['To Bank']}
    - Receiver Account: {transaction['Account.1']}
    - Amount Paid: {transaction['Amount Paid']} {transaction['Payment Currency']}
    - Amount Received: {transaction['Amount Received']} {transaction['Receiving Currency']}
    - Payment Format: {transaction['Payment Format']}

    You are an AML expert.
    Based on the transaction details above, rate the risk of money laundering as a number between 0 and 1.

    **ONLY RETURN A SINGLE FLOATING NUMBER. DO NOT PROVIDE ANY EXPLANATION.**
    """

    try:
        response = client.chat.completions.create(
            model="deepseek-reasoner",
            messages=[{"role": "system", "content": "You are an AML risk expert."},
                      {"role": "user", "content": prompt}],
            max_tokens=5,
            temperature=0.2,
            stream=False
        )

        raw_output = response.choices[0].message.content.strip()
        score = max(0.0, min(float(raw_output), 1.0))  # Ensure score is between 0-1
        return score
    except Exception as e:
        print(f"API Call Failed: {e}")
        return 0.5  # Default if API fails

# Process transactions one by one
for index, row in df.iterrows():
    transaction_id = row["Transaction ID"]

    # Skip if already processed
    if transaction_id in processed_ids:
        continue

    # Get DeepSeek Score
    deepseek_score = get_deepseek_score(row)

    # Save incrementally
    processed_scores = pd.concat([processed_scores, pd.DataFrame({"Transaction ID": [transaction_id], "DeepSeek Score": [deepseek_score]})], ignore_index=True)
    processed_scores.to_csv(deepseek_score_path, index=False)

    # Print progress
    print(f"Processed Transaction {transaction_id} | DeepSeek Score: {deepseek_score:.4f}")

print("All transactions processed and saved to deepseek_scores.csv")


<ipython-input-6-09c032b0bd83>:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processed_scores = pd.concat([processed_scores, pd.DataFrame({"Transaction ID": [transaction_id], "DeepSeek Score": [deepseek_score]})], ignore_index=True)


✅ Processed Transaction 0 | DeepSeek Score: 0.1000
✅ Processed Transaction 1 | DeepSeek Score: 0.0100
✅ Processed Transaction 2 | DeepSeek Score: 0.1000
✅ Processed Transaction 3 | DeepSeek Score: 0.0000
✅ Processed Transaction 4 | DeepSeek Score: 0.2000
✅ Processed Transaction 5 | DeepSeek Score: 0.1000
✅ Processed Transaction 6 | DeepSeek Score: 0.0000
✅ Processed Transaction 7 | DeepSeek Score: 0.0000
✅ Processed Transaction 8 | DeepSeek Score: 0.1000
✅ Processed Transaction 9 | DeepSeek Score: 0.2000
✅ Processed Transaction 10 | DeepSeek Score: 0.1000
✅ Processed Transaction 11 | DeepSeek Score: 0.0000
✅ Processed Transaction 12 | DeepSeek Score: 0.1000
✅ Processed Transaction 13 | DeepSeek Score: 0.1000
✅ Processed Transaction 14 | DeepSeek Score: 0.0100
✅ Processed Transaction 15 | DeepSeek Score: 0.7000
✅ Processed Transaction 16 | DeepSeek Score: 0.2000
✅ Processed Transaction 17 | DeepSeek Score: 0.1000
✅ Processed Transaction 18 | DeepSeek Score: 0.1000
✅ Processed Transactio

Model Training

In [ ]:
import os
import torch
import networkx as nx
from torch_geometric.utils import from_networkx
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import os
import time

# Load transaction dataset
file_path = "/content/drive/MyDrive/285project/train.csv"
df = pd.read_csv(file_path)

# Load DeepSeek scores
deepseek_file = "/content/drive/MyDrive/285project/deepseek_scores.csv"
if os.path.exists(deepseek_file):
    deepseek_df = pd.read_csv(deepseek_file)
    df = df.merge(deepseek_df, on="Transaction ID", how="left")
else:
    raise FileNotFoundError("DeepSeek scores not found! Run DeepSeek scoring first.")

# Normalize Timestamp and Amounts
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["Timestamp"] = (df["Timestamp"] - df["Timestamp"].min()).dt.total_seconds()
scaler = MinMaxScaler()
df[["Amount Paid", "Amount Received"]] = scaler.fit_transform(df[["Amount Paid", "Amount Received"]])

# Convert categorical variables
df["Receiving Currency"] = df["Receiving Currency"].astype("category").cat.codes
df["Payment Currency"] = df["Payment Currency"].astype("category").cat.codes
df["Payment Format"] = df["Payment Format"].astype("category").cat.codes

# Create Graph
G_nx = nx.from_pandas_edgelist(
    df,
    source="Account",
    target="Account.1",
    edge_attr=["Timestamp", "Amount Paid", "Amount Received", "Receiving Currency", "Payment Currency", "Payment Format", "DeepSeek Score", "Is Laundering"],
    create_using=nx.DiGraph()
)

# Convert to PyTorch Geometric Graph
G_pyg = from_networkx(G_nx, group_node_attrs=["Timestamp", "Amount Paid", "Amount Received", "Receiving Currency", "Payment Currency", "Payment Format"])
G_pyg.edge_attr = torch.tensor(df["DeepSeek Score"].values, dtype=torch.float32).view(-1, 1)
G_pyg = G_pyg.to("cpu")


In [ ]:
import gym
import numpy as np
import torch
from stable_baselines3 import PPO
from tqdm.notebook import tqdm

class AML_PyG_Env(gym.Env):
    def __init__(self, graph):
        super(AML_PyG_Env, self).__init__()
        self.graph = graph
        self.index = 0

        self.action_space = gym.spaces.Discrete(2)  # 0: Normal, 1: Suspicious
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(graph.x.shape[1],), dtype=np.float32)  # FIXED

    def reset(self):
        self.index = 0
        return self.graph.x[self.index].cpu().numpy()

    def step(self, action):
        node_id = self.index
        transaction = self.graph.x[node_id]
        true_label = self.graph.edge_attr[node_id].item()  # 0 (Not Fraud) or 1 (Fraud)
        deepseek_score = self.graph.edge_attr[node_id].item()  # Use precomputed DeepSeek Score

        if action == true_label:
            reward = 1 + deepseek_score  # Higher reward if action matches label
        else:
            reward = -1 + deepseek_score  # Penalize wrong predictions

        self.index += 1
        done = self.index >= self.graph.num_nodes
        next_state = self.graph.x[self.index].cpu().numpy() if not done else None

        return next_state, reward, done, {}


In [ ]:
import gym
import numpy as np
import torch
from stable_baselines3 import PPO
from tqdm.notebook import tqdm

# Define PPO Training Environment
env = AML_PyG_Env(G_pyg)

# Train PPO Model on CPU
TOTAL_TIMESTEPS = 50000
model = PPO("MlpPolicy", env, verbose=0, device="cpu")

with tqdm(total=TOTAL_TIMESTEPS, desc="Training Progress", unit="steps") as pbar:
    for _ in range(TOTAL_TIMESTEPS // 10000):
        model.learn(total_timesteps=10000, reset_num_timesteps=False)
        print(f" Step: {env.index} | Last DeepSeek Score: {env.graph.edge_attr[env.index - 1].item():.4f}")
        pbar.update(10000)

# Save the trained model
model.save("/content/drive/MyDrive/285project/ppo_aml")
print(" PPO Model Saved: /mnt/data/ppo_aml.zip")


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Training Progress:   0%|          | 0/50000 [00:00<?, ?steps/s]

✅ Step: 22 | Last DeepSeek Score: 0.0000
✅ Step: 44 | Last DeepSeek Score: 0.1500
✅ Step: 66 | Last DeepSeek Score: 0.5000
✅ Step: 10 | Last DeepSeek Score: 0.2000
✅ Step: 32 | Last DeepSeek Score: 0.1000
🎉 PPO Model Saved: /mnt/data/ppo_aml.zip


Model Testing

In [ ]:
import pandas as pd
import torch
import networkx as nx
from torch_geometric.utils import from_networkx
from stable_baselines3 import PPO
import numpy as np

# Load Test Dataset
test_file_path = "/content/drive/MyDrive/285project/test.csv"  # Ensure correct path
df_test = pd.read_csv(test_file_path)

# Normalize Timestamp (convert to seconds)
df_test["Timestamp"] = pd.to_datetime(df_test["Timestamp"], errors="coerce")
df_test["Timestamp"] = (df_test["Timestamp"] - df_test["Timestamp"].min()).dt.total_seconds()

# Convert categorical variables to numeric codes
df_test["Receiving Currency"] = df_test["Receiving Currency"].astype("category").cat.codes
df_test["Payment Currency"] = df_test["Payment Currency"].astype("category").cat.codes
df_test["Payment Format"] = df_test["Payment Format"].astype("category").cat.codes

# Normalize numerical features (Amount Sent & Amount Received)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_test[["Amount Received", "Amount Paid"]] = scaler.fit_transform(df_test[["Amount Received", "Amount Paid"]])

# Create Graph from Test Data
G_nx_test = nx.from_pandas_edgelist(
    df_test,
    source="Account",
    target="Account.1",
    edge_attr=["Timestamp", "Amount Received", "Amount Paid", "Receiving Currency", "Payment Currency", "Payment Format"],
    create_using=nx.DiGraph()
)

# Convert to PyTorch Geometric Graph
G_pyg_test = from_networkx(G_nx_test, group_node_attrs=["Timestamp", "Amount Received", "Amount Paid", "Receiving Currency", "Payment Currency", "Payment Format"])
print("Test Dataset Converted to PyG Graph")

# Load the trained PPO model
model = PPO.load("/content/drive/MyDrive/285project/ppo_aml")
print("PPO Model Loaded Successfully!")

# Define Test Environment (test_mode=True prevents label checks)
class AML_PyG_Env(gym.Env):
    def __init__(self, graph, test_mode=False):
        super(AML_PyG_Env, self).__init__()
        self.graph = graph
        self.index = 0
        self.test_mode = test_mode

        self.action_space = gym.spaces.Discrete(2)  # 0: Normal, 1: Suspicious
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(graph.x.shape[1],), dtype=np.float32)

    def reset(self):
        self.index = 0
        return self.graph.x[self.index].cpu().numpy()

    def step(self, action):
        node_id = self.index
        transaction = self.graph.x[node_id]

        if self.graph.edge_attr is not None:
            deepseek_score = self.graph.edge_attr[node_id].item()
        else:
            deepseek_score = 0.5  # Default score if missing

        reward = deepseek_score  # No label checking in test mode

        self.index += 1
        done = self.index >= self.graph.num_nodes
        next_state = self.graph.x[self.index].cpu().numpy() if not done else None

        return next_state, reward, done, {}

# Run PPO Model on Test Transactions
test_env = AML_PyG_Env(G_pyg_test, test_mode=True)
predictions = []

for i in range(len(G_pyg_test.x)):  # Iterate through test transactions
    state = test_env.reset()
    done = False

    while not done:
        action, _ = model.predict(state, deterministic=True)  # Get model prediction
        state, _, done, _ = test_env.step(action)  # Step through env

    # Store prediction results
    predictions.append({"Transaction ID": i, "Predicted Label": action})

    print(f"Test Transaction {i} | Predicted Label: {'Suspicious' if action == 1 else 'Normal'}")

# Save Predictions to CSV
pred_df = pd.DataFrame(predictions)
pred_df.to_csv("/content/drive/MyDrive/285project/test_predictions.csv", index=False)

print("🎉 Test Predictions Saved: /content/drive/MyDrive/285project/test_predictions.csv")


✅ Test Dataset Converted to PyG Graph
✅ PPO Model Loaded Successfully!
🔍 Test Transaction 0 | Predicted Label: Normal
🔍 Test Transaction 1 | Predicted Label: Normal
🔍 Test Transaction 2 | Predicted Label: Normal
🔍 Test Transaction 3 | Predicted Label: Normal
🔍 Test Transaction 4 | Predicted Label: Normal
🔍 Test Transaction 5 | Predicted Label: Normal
🔍 Test Transaction 6 | Predicted Label: Normal
🔍 Test Transaction 7 | Predicted Label: Normal
🔍 Test Transaction 8 | Predicted Label: Normal
🔍 Test Transaction 9 | Predicted Label: Normal
🔍 Test Transaction 10 | Predicted Label: Normal
🔍 Test Transaction 11 | Predicted Label: Normal
🔍 Test Transaction 12 | Predicted Label: Normal
🔍 Test Transaction 13 | Predicted Label: Normal
🔍 Test Transaction 14 | Predicted Label: Normal
🔍 Test Transaction 15 | Predicted Label: Normal
🔍 Test Transaction 16 | Predicted Label: Normal
🔍 Test Transaction 17 | Predicted Label: Normal
🎉 Test Predictions Saved: /content/drive/MyDrive/285project/test_prediction